# 作業 : (Kaggle)鐵達尼生存預測
https://www.kaggle.com/c/titanic

# [作業目標]
- 試著模仿範例寫法, 在鐵達尼生存預測中, 觀察標籤編碼與獨編碼熱的影響

# [作業重點]
- 回答在範例中的觀察結果
- 觀察標籤編碼與獨熱編碼, 在特徵數量 / 邏輯斯迴歸分數 / 邏輯斯迴歸時間上, 分別有什麼影響 (In[3], Out[3], In[4], Out[4]) 

# 作業1
* 觀察範例，在房價預測中調整標籤編碼(Label Encoder) / 獨熱編碼 (One Hot Encoder) 方式，  
對於線性迴歸以及梯度提升樹兩種模型，何者影響比較大?

#### Answer : 線性迴歸模型；score 從 0.662 掉至 -5.720。

# 作業2
* 鐵達尼號例題中，標籤編碼 / 獨熱編碼又分別對預測結果有何影響? (Hint : 參考今日範例)

#### Answer : 對於迴歸模型而言，獨熱編碼能提升模型的表現，但同時須花費較多的運算成本。 

In [1]:
# 做完特徵工程前的所有準備 (與前範例相同)
import pandas as pd
import numpy as np
import copy, time
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

data_path = '/Users/YenLin/Desktop/ML_100/data/'
df_train = pd.read_csv( data_path + 'titanic_train.csv' )
df_test = pd.read_csv( data_path + 'titanic_test.csv' )

train_Y = df_train[ 'Survived' ]
ids = df_test[ 'PassengerId' ]
df_train = df_train.drop( [ 'PassengerId', 'Survived' ] , axis = 1 )
df_test = df_test.drop( [ 'PassengerId' ] , axis = 1 )
df = pd.concat( [ df_train, df_test ] )
df.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
#只取類別值 (object) 型欄位, 存於 object_features 中
object_features = []
for dtype, feature in zip( df.dtypes, df.columns ) :
    if dtype == 'object':
        object_features.append( feature )
print(f'{len(object_features)} Numeric Features : {object_features}\n')

# 只留類別型欄位
df = df[object_features]
df = df.fillna('None')
train_num = train_Y.shape[0]
df.head()

5 Numeric Features : ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']



,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,None,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,None,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,None,S


In [3]:
# 標籤編碼 + 羅吉斯迴歸

df_temp_1 = pd.DataFrame( )  # 創造一個空的 Dataframe

# Label Encoding
for c in df.columns :
    df_temp_1[ c ] = LabelEncoder( ).fit_transform( df[ c ] )

# 羅吉斯迴歸模型
estimator = LogisticRegression( solver = 'liblinear' )
    
train_X = df_temp_1[ : train_num ]

start = time.time()
print( 'Time = ' + str( time.time() - start ) + 'sec' )
print( 'Shape of Training Data = ' + str( train_X.shape ) )
print( 'Score = ' + str( cross_val_score( estimator, train_X, train_Y, cv = 5 ).mean( ) ) + '\n' )

train_X.head( 15 )

Time = 4.8160552978515625e-05sec
Shape of Training Data = (891, 5)
Score = 0.780004837244799



,Name,Sex,Ticket,Cabin,Embarked
0,155,1,720,185,3
1,286,0,816,106,0
2,523,0,914,185,3
3,422,0,65,70,3
4,22,1,649,185,3
5,818,1,373,185,2
6,767,1,109,163,3
7,914,1,541,185,3
8,605,0,477,185,3
9,847,0,174,185,0


In [4]:
# 獨熱編碼 + 羅吉斯迴歸

# One-Hot Encoding
df_temp_2 = pd.get_dummies( df )

# 羅吉斯迴歸模型
estimator = LogisticRegression( solver = 'liblinear' )

train_X = df_temp_2[ : train_num ]

start = time.time()
print( 'Time = ' + str( time.time() - start ) + 'sec' )
print( 'Shape of Training Data = ' + str( train_X.shape ) )
print( 'Score = ' + str( cross_val_score( estimator, train_X, train_Y, cv = 5 ).mean( ) ) + '\n' )

train_X.head( 15 )

Time = 7.581710815429688e-05sec
Shape of Training Data = (891, 2429)
Score = 0.8013346043513216



,"Name_Abbing, Mr. Anthony","Name_Abbott, Master. Eugene Joseph","Name_Abbott, Mr. Rossmore Edward","Name_Abbott, Mrs. Stanton (Rosa Hunt)","Name_Abelseth, Miss. Karen Marie","Name_Abelseth, Mr. Olaus Jorgensen","Name_Abelson, Mr. Samuel","Name_Abelson, Mrs. Samuel (Hannah Wizosky)","Name_Abrahamsson, Mr. Abraham August Johannes","Name_Abrahim, Mrs. Joseph (Sophie Halaut Easu)",...,Cabin_F33,Cabin_F38,Cabin_F4,Cabin_G6,Cabin_None,Cabin_T,Embarked_C,Embarked_None,Embarked_Q,Embarked_S
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
